<a href="https://colab.research.google.com/github/Funky-Synatra/Literary_Style_Models/blob/main/Copy_of_Literary_Style_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Colab Notebook

# Etapa 1: Configuração do ambiente
!pip install transformers datasets

import os
import torch
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# Verificar se GPU está disponível
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Funções para carregar o dataset e o data collator
def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )
    return dataset

def get_data_collator(tokenizer):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

# Etapa 2: Configuração para diferentes línguas e estilos
languages = ['portuguese', 'english']
styles = ['comedy', 'drama', 'romance']
file_paths = {
    'portuguese': {
        'comedy': "/content/portuguese_comedy.txt",
        'drama': "/content/portuguese_drama.txt",
        'romance': "/content/portuguese_romance.txt"
    },
    'english': {
        'comedy': "/content/english_comedy.txt",
        'drama': "/content/english_drama.txt",
        'romance': "/content/english_romance.txt"
    }
}
model_paths = {
    'portuguese': {
        'comedy': "./results_portuguese_comedy",
        'drama': "./results_portuguese_drama",
        'romance': "./results_portuguese_romance"
    },
    'english': {
        'comedy': "./results_english_comedy",
        'drama': "./results_english_drama",
        'romance': "./results_english_romance"
    }
}

# Carregar o tokenizador
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Função para treinar o modelo em um estilo específico
def train_model(language, style):
    print(f"Treinando modelo para {language} - {style}...")

    # Carregar o dataset específico da língua e estilo
    new_dataset = load_dataset(file_paths[language][style], tokenizer)

    # Verificar se o modelo já existe
    model_path = model_paths[language][style]
    if os.path.exists(model_path):
        print(f"Carregando modelo existente para {language} - {style}...")
        model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
    else:
        print(f"Iniciando um novo modelo do zero para {language} - {style}...")
        config = GPT2Config()
        model = GPT2LMHeadModel(config).to(device)

    # Definir argumentos de treinamento
    training_args = TrainingArguments(
        output_dir=model_path,
        overwrite_output_dir=True,
        num_train_epochs=5,  # Pode ajustar o número de épocas
        per_device_train_batch_size=2,  # Pode ajustar dependendo dos recursos disponíveis
        save_steps=10_000,
        save_total_limit=2,
    )

    # Continuar o treinamento com o novo dataset
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=get_data_collator(tokenizer),
        train_dataset=new_dataset,
    )

    # Iniciar o treinamento com novos dados
    trainer.train()

    # Salvar o modelo após o treinamento
    model.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)

    print(f"Treinamento do modelo para {language} - {style} concluído.")

# Treinar modelos para cada combinação de língua e estilo
for language in languages:
    for style in styles:
        train_model(language, style)

# Função para gerar texto com um modelo específico
def generate_text(prompt, language, style, max_length=100):
    model_path = model_paths[language][style]
    model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)

    model.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(inputs.input_ids, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Testar o gerador de texto para diferentes línguas e estilos
prompt = "Era uma vez"
for language in languages:
    for style in styles:
        print(f"Texto gerado para {language} - {style}:")
        generated_text = generate_text(prompt, language, style)
        print(generated_text)
        print("\n" + "="*50 + "\n")
